# Analisi della Correlazione tra Sentiment delle Notizie e Prezzo di Bitcoin

**Nome:** Alessandro Zucchiatti
**Corso:** Fondamenti di Analisi Dati e Laboratorio
**Progetto:** Analisi del sentiment e dell'andamento di prezzo di Bitcoin

### Obiettivo
Investigare la relazione tra il sentiment delle notizie su Bitcoin e l'andamento del prezzo giornaliero tramite feature engineering e modelli statistici.

**Domanda di ricerca:** Esiste una correlazione statisticamente significativa tra il sentiment aggregato giornaliero delle notizie e la variazione percentuale del prezzo di Bitcoin del giorno successivo?

### Setup e Import Librerie

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import yfinance as yf
import ast
import kagglehub
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.stattools import grangercausalitytests

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 7)
plt.rcParams['figure.dpi'] = 100
plt.style.use('dark_background') 

plt.rcParams.update({
    "font.size": 11,                 # testo leggermente più grande
    "axes.titleweight": "bold",
    "axes.edgecolor": "#444444",
    "grid.color": "#555555",
    "grid.alpha": 0.3,
})

print("Librerie importate con successo!")

### Caricamento e Preparazione dei Dati

In [ ]:
# Caricamento dataset sentiment
path = kagglehub.dataset_download('oliviervha/crypto-news')
file_path = path + "/cryptonews.csv"

try:
    sentiment_df = pd.read_csv(file_path)
    print("Dataset del sentiment caricato.")
    display(sentiment_df.head())
except FileNotFoundError:
    print(f"Errore: File '{file_path}' non trovato.")

In [ ]:
# Filtraggio e pulizia dati sentiment
btc_sentiment_df = sentiment_df[sentiment_df['subject'] == 'bitcoin'].copy()

def parse_sentiment(sentiment_str):
    try:
        return ast.literal_eval(sentiment_str)
    except (ValueError, SyntaxError):
        return {'class': 'neutral', 'polarity': 0.0, 'subjectivity': 0.0}

sentiment_dicts = btc_sentiment_df['sentiment'].apply(parse_sentiment)
btc_sentiment_df['sentiment_class'] = [d.get('class', 'neutral') for d in sentiment_dicts]
btc_sentiment_df['polarity'] = [d.get('polarity', 0.0) for d in sentiment_dicts]
btc_sentiment_df['subjectivity'] = [d.get('subjectivity', 0.0) for d in sentiment_dicts]
btc_sentiment_df['date'] = pd.to_datetime(btc_sentiment_df['date'])
btc_sentiment_df = btc_sentiment_df[['date', 'sentiment_class', 'polarity', 'subjectivity']]

print("Pulizia dati sentiment completata.")
display(btc_sentiment_df.head())

In [ ]:
# Caricamento dati prezzo Bitcoin
start_date = btc_sentiment_df['date'].min().strftime('%Y-%m-%d')
end_date = (btc_sentiment_df['date'].max() + pd.Timedelta(days=1)).strftime('%Y-%m-%d')

btc_price_df = yf.download('BTC-USD', start=start_date, end=end_date)
btc_price_df['pct_change'] = btc_price_df['Close'].pct_change() * 100
btc_price_df = btc_price_df.reset_index().rename(columns={'Date': 'date'})

print(f"Dati prezzo Bitcoin scaricati dal {start_date} al {end_date}.")
display(btc_price_df.head())

### Aggregazione e Feature Engineering

In [ ]:
# Aggregazione sentiment su base giornaliera
btc_sentiment_df.set_index('date', inplace=True)
daily_sentiment = btc_sentiment_df.resample('D').agg(
    polarity_mean=('polarity', 'mean'),
    polarity_std=('polarity', 'std'),
    subjectivity_mean=('subjectivity', 'mean'),
    news_count=('polarity', 'count')
)
daily_sentiment['polarity_std'] = daily_sentiment['polarity_std'].fillna(0)
daily_sentiment = daily_sentiment[daily_sentiment['news_count'] > 0].reset_index()

print("Aggregazione giornaliera completata.")
display(daily_sentiment.head())

### Unione dei Dati

In [ ]:
# Merge dei dataset
daily_sentiment['date'] = daily_sentiment['date'].dt.tz_localize(None)

if isinstance(btc_price_df.columns, pd.MultiIndex):
    btc_price_df.columns = btc_price_df.columns.droplevel(1)

btc_price_df['date'] = pd.to_datetime(btc_price_df['date'])
df_merged = pd.merge(btc_price_df, daily_sentiment, on='date', how='inner')
df_merged = df_merged.dropna().set_index('date')

print(f"Merge completato: {len(df_merged)} righe")
display(df_merged.head())

### Analisi Esplorativa e Visualizzazione

In [ ]:
# Andamento prezzo vs polarità
fig, ax1 = plt.subplots()
ax1.plot(
    df_merged.index,
    df_merged['Close'],
    color='#9ef01a',            # lime
    linewidth=2.2,
    marker='o',
    markersize=3,
    label='Prezzo BTC'
)

ax1.set_xlabel('Data')
ax1.set_ylabel('Prezzo (USD)', color='#9ef01a')
ax1.tick_params(axis='y', labelcolor='#9ef01a')

# Date su due righe “Mese / Anno”
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
ax1.grid(True, linestyle='--', linewidth=0.6, alpha=0.3)

ax2 = ax1.twinx()
ax2.plot(
    df_merged.index,
    df_merged['polarity_mean'].rolling(window=7).mean(),
    color='#e600ac',            # magenta
    linewidth=2,
    label='Polarità (Media 7gg)'
)

ax2.set_ylabel('Polarità Media', color='#e600ac')
ax2.tick_params(axis='y', labelcolor='#e600ac')
ax2.axhline(0, color='#888888', linestyle='--', linewidth=1)

handles, labels = [], []
for ax in (ax1, ax2):
    h, l = ax.get_legend_handles_labels()
    handles.extend(h)
    labels.extend(l)

ax1.legend(
    handles, labels,
    loc='upper left',
    frameon=True,
    facecolor='#222222',
    framealpha=0.8
)

plt.title('Prezzo Bitcoin vs. Polarità Media Notizie')
fig.tight_layout()
plt.show()


In [ ]:
# Matrice di correlazione
correlation_cols = ['Close', 'Volume', 'pct_change', 'polarity_mean', 'polarity_std', 'subjectivity_mean', 'news_count']
correlation_matrix = df_merged[correlation_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Matrice di Correlazione')
plt.show()

In [ ]:
# Scatter plot polarità vs variazione prezzo
sns.regplot(x='polarity_mean', y='pct_change', data=df_merged, 
            scatter_kws={'alpha':0.3, 'color': 'royalblue'}, 
            line_kws={'color':'red', 'linestyle':'--'})
plt.title('Variazione % Prezzo vs. Polarità Media')
plt.xlabel('Polarità Media')
plt.ylabel('Variazione % Prezzo')
plt.show()

### Modellazione Predittiva

In [ ]:
# Preparazione dati per modello predittivo
df_model = df_merged.copy()
df_model['target_pct_change'] = df_model['pct_change'].shift(-1)
df_model = df_model.dropna()

features = ['polarity_mean', 'polarity_std', 'subjectivity_mean', 'news_count', 'Volume', 'pct_change']
X = df_model[features]
y = df_model['target_pct_change']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
print(f"Dataset: {X_train.shape[0]} train, {X_test.shape[0]} test")

In [ ]:
# Regressione lineare
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Regressione Lineare - MSE: {mse:.4f}, R²: {r2:.4f}")

coeffs = pd.DataFrame(model.coef_, X.columns, columns=['Coefficiente']).sort_values('Coefficiente', ascending=False)
print("\nCoefficienti del modello:")
display(coeffs)

print("\nUn R² basso indica che il modello lineare non spiega efficacemente la variabilità del prezzo.")

In [ ]:
# Test di Causalità di Granger
causality_df = df_merged[['pct_change', 'polarity_mean']].dropna()
max_lag = 7

print(f"Test di Causalità di Granger (max_lag={max_lag})")

try:
    granger_test_result = grangercausalitytests(causality_df, maxlag=max_lag)
    
    p_values = [round(granger_test_result[lag][0]['ssr_ftest'][1], 4) for lag in range(1, max_lag + 1)]
    print(f"\nP-values per lag 1-{max_lag}: {p_values}")
    
    if any(p < 0.05 for p in p_values):
        print("\nRisultato: Evidenza di Causalità di Granger trovata.")
    else:
        print("\nRisultato: Nessuna evidenza di Causalità di Granger.")

except Exception as e:
    print(f"\nErrore test di Granger: {e}")

### Conclusioni

**Risultati principali:**
1. La correlazione diretta tra sentiment e variazione di prezzo nello stesso giorno è molto debole
2. Il modello di regressione lineare mostra performance limitata (basso R²)
3. Il test di Causalità di Granger valuta se il sentiment passato ha valore predittivo

**Considerazioni:** Il mercato Bitcoin è influenzato da molteplici fattori complessi che il solo sentiment delle notizie non riesce a catturare completamente.
